In [43]:
palette = ['#3B8EA1', '#B84D4A', '#6F5B9A', '#B59F8E', '#D96B9A', '#A3A3A3', '#D88F3B', '#4C9F4A']
from pathlib import Path
enrich_output = Path("..") / "data" / "raw_scores"
mrna_data = Path("..") / "data" / "other_data" / "mrna_data"
save = Path("..") / "data" / "processed_scores"
figure = Path("..") / "figures"
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [21]:
def barcode_to_variant(experiment, barcode_variant_table, enrich_scores_PATH):
    df_score = pd.read_csv(f"{enrich_scores_PATH}\\{experiment}_main_barcodes_scores.tsv", sep='\t').rename(
        columns={"Unnamed: 0": "barcode"})
    df_score = df_score[["barcode", "score"]]
    df_barvar = pd.read_csv(f"{enrich_scores_PATH}\\{barcode_variant_table}.csv")
    df_merge = df_barvar.merge(df_score, left_on="barcode_rev", right_on="barcode").drop(columns=["barcode", "barcode_rev"])
    df = df_merge[(df_merge["n_aa_substitutions"] <= 1)]
    df = df.rename(columns={"aa_substitutions": "variant", "codon_substitutions": "codon"})
    return df[["variant", "codon", "n_aa_substitutions", "n_codon_substitutions", "variant_call_support", "score"]]

In [36]:
def variant_to_type(df):
    df["sequence_type"] = np.where(df["n_aa_substitutions"] == 1, "single_variant", "wild_type")

    # Conditions for assigning variant_type
    conditions = [
        df["variant"].str.contains(r"^\*", na=False),  # for stop_loss (variant starts with *)
        df["variant"].str.contains(r"\*$", na=False),  # for stop_gain (variant ends with *)
        (df["sequence_type"] == "wild_type") & (df["n_codon_substitutions"] == 0), # unmutated
        (df["sequence_type"] == "wild_type") & (df["n_codon_substitutions"] != 0), # synonymous
        (df["sequence_type"] != "wild_type") & (df["variant"].str.contains(r"\*", na=False)) # undefined
    ]

    values = ["stop_loss", "stop_gain", "unmutated", "synonymous", "undefined"]

    # Assign variant_type
    df["variant_type"] = np.select(conditions, values, default="missense")
    
    # Keep all non-unmutated, and only well-supported unmutated (support >= 3)
    # Also ensure undefined variants have sufficient support if they are to be included
    df_process = pd.concat([
        df[df["variant_type"] != "unmutated"],
        df[(df["variant_type"] == "unmutated") & (df["variant_call_support"] >= 3)]
    ], axis=0).reset_index(drop=True)

    # Drop intermediate columns after all logic that uses them
    df_process.drop(columns=["n_codon_substitutions", "n_aa_substitutions"], inplace=True)
    return df_process

In [37]:
def normalize_to_growth(df, scale=(0, 1)):
    stop_gain = df.loc[df["variant_type"] == "stop_gain", "score"].median()
    wildtype = df.loc[df["variant_type"] == "unmutated", "score"].median()
    print(f"stop_gain: {round(stop_gain,2)} --> {scale[0]}")
    print(f"wild_type: {round(wildtype,2)} --> {scale[1]}")
    # Linear rescaling between stop_gain and wildtype
    df["norm_score"] = ((df["score"] - stop_gain) / (wildtype - stop_gain)) * (scale[1] - scale[0]) + scale[0]
    barcode_stdv = df.groupby('variant')['score'].std().mean()
    stdv = df['score'].std()
    print(f"exp_err: {round(barcode_stdv/stdv,3)}")
    return df

In [38]:
def multiple_to_single(df):
    # ----------------- Variant-based group -----------------
    df_wt = df[df["sequence_type"] == "wild_type"].copy()
    df_wt['codon'] = df_wt['codon'].fillna("wild-type")
    df_wt = df_wt[["codon", "norm_score", "variant_call_support", "sequence_type", "variant_type"]]
    df_wt = df_wt.rename(columns={"codon": "variant"})
    # Filter non-wild type data
    df_single = df[df["sequence_type"] != "wild_type"][
        ["variant", "norm_score", "variant_call_support", "sequence_type", "variant_type"]]
    dfOne = pd.concat([df_single, df_wt], axis=0).reset_index(drop=True)
    variant_types = ["missense", "stop_gain", "synonymous", "unmutated", "stop_loss"]
    processed_variant = []
    for var_type in variant_types:
        filtered_df = dfOne[dfOne["variant_type"] == var_type]
        grouped = filtered_df.groupby('variant')
        # Aggregating metrics
        fitness_score = grouped['norm_score'].median()
        iqr_score = grouped['norm_score'].apply(lambda x: x.quantile(0.75) - x.quantile(0.25))
        sem = grouped['norm_score'].sem()
        group_counts = grouped.size()
        aggregated_variant = pd.DataFrame({
            'variant': fitness_score.index,
            'fitness_score': fitness_score.values,
            'iqr': iqr_score.values,
            'sem': sem.values,
            'barcodes': group_counts
        })
        
        # Add 'low_barcode_warning' column
        aggregated_variant['warning'] = aggregated_variant['barcodes'].apply(lambda x: "low_confidence" if x <= 3 else "")
        aggregated_variant['sequence_type'] = 'single_variant' if var_type in ["missense", "stop_gain", "stop_loss"] else 'wild_type'
        aggregated_variant['variant_type'] = var_type
        processed_variant.append(aggregated_variant)
    
    df_variant = pd.concat(processed_variant, axis=0).reset_index(drop=True)
    # Split 'variant' into components
    split = df_variant["variant"].str.split(r'(\d+)', n=1, expand=True).rename(columns={0: "residue", 1: "position", 2: "change"})
    df_variant = pd.concat([split, df_variant], axis=1)
    df_variant = df_variant[[ "variant", "residue", "position", "change", "fitness_score", "iqr", "sem", "barcodes", 
                              "warning", "sequence_type", "variant_type"]]
    df_variant['position'] = df_variant['position'].fillna(0) #for wildtype
    df_variant['position'] = pd.to_numeric(df_variant['position'], errors='coerce').astype(int)
    df_variant.sort_values(by='position', ascending=True, inplace=True)
    return df_variant

In [50]:
condition = "dox-"
exp = "highADSL_control" # lowADSL-T7, lowADSL_control, highADSL-T7, highADSL_control
one = barcode_to_variant(f"{exp}", "highADSL_barcode_variantR", enrich_output)
two = variant_to_type(one)
three = normalize_to_growth(two, [1.27, 1.38]) #dox (low, high) [0-1], [0-1.35], control [1.27-1.33], [1.27-1.38]
df = multiple_to_single(three)
df.to_csv(f"{save}\\{exp[:7]}_{condition}_fitness_score.csv", index = False)
print("data saved..")

stop_gain: -0.33 --> 1.27
wild_type: 0.07 --> 1.38
exp_err: 0.874
data saved..
